In [1]:
%nvidia-smi

UsageError: Line magic function `%nvidia-smi` not found.


In [19]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from PIL import Image
import time
from skimage.metrics import peak_signal_noise_ratio

In [3]:
# CPU/GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available.')

cuda:0 is available.


In [4]:
class MyDataset(Dataset):
    def __init__(self, image_dir, transform=None, test=False):
        self.image_dir = image_dir
        self.image_list = os.listdir(os.path.join(image_dir, 'blur/'))
        self.image_list.sort()
        self.transform = transform
        self.test = test

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.image_dir, 'blur', self.image_list[idx]))
        label = Image.open(os.path.join(self.image_dir, 'sharp', self.image_list[idx]))

        if self.transform:
            image = self.transform(image)
            label = self.transform(label)
        else:
            tf_toTensor = transforms.ToTensor()
            image = tf_toTensor(image)
            label = tf_toTensor(label)
        if self.test:
            name = self.image_list[idx]
            return image, label, name
        return image, label

In [5]:
def train_dataloader(path, batch_size = 64, num_workers=0):
    image_dir = os.path.join(path, 'train')

    transform = transforms.Compose(
        
        [transforms.Resize((256, 256)),
         transforms.RandomCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         ]
     )
  
    dataloader = DataLoader(
        MyDataset(image_dir, transform=transform),
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
        )
    return dataloader

In [6]:
def test_dataloader(path, batch_size = 1, num_workers=0):
    image_dir = os.path.join(path, 'test')
  
    dataloader = DataLoader(
        MyDataset(image_dir, test=True),
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers
        )
    return dataloader

In [7]:
path = os.getcwd()

In [16]:
def _train(model, max_epoch = 10, lr = 0.002, resume=False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    criterion = torch.nn.L1Loss()
    optimizer = torch.optim.Adam(net.parameters(), lr, weight_decay=0)
    dataloader = train_dataloader(path, batch_size=32, num_workers=0)
    writer = SummaryWriter('logs/')

    epoch = 1

    if (resume == True):
        PATH = 'weights/weights0.002v3.pth'
        checkpoint = torch.load(PATH)
        net.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        epoch += 1
    print('Starting from Epoch :',epoch)

    since = time.time()
    for epoch_idx in range(epoch, max_epoch+1):
        running_loss = 0.0
        cnt = 0
        epoch_time = time.time()
        for idx, batch in enumerate(dataloader):
            optimizer.zero_grad()
            input, target = batch
            input = input.to(device)
            target = target.to(device)

            output = net(input)
            loss = criterion(output, target)
            writer.add_scalar("Loss/train", loss, epoch_idx)
                
            loss.backward()
            optimizer.step()
  
            running_loss += loss.item()
            cnt += 1

        if (epoch_idx % 10 == 1):
            time_elapsed = time.time() - epoch_time
            total_time = time.time() - since
            print('Training complete in {:.0f}m {:.0f}s, Total time : {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60, total_time // 60, total_time % 60))
            print(f'Epoch: {epoch_idx}, Loss: {running_loss / cnt:.3f}saved')
      
        PATH = 'weights/weights0.002v3.pth'
        torch.save({
                'epoch': epoch_idx,
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                
                  }, PATH)
    writer.flush()
    writer.close()
    print('Finished Training')

In [15]:
def _test(model):
    PATH = 'weights/weights0.002v3.pth'
    checkpoint = torch.load(PATH)
    net.load_state_dict(checkpoint['model_state_dict'])

    dataloader = test_dataloader(path, batch_size=1, num_workers=0)
    torch.cuda.empty_cache()
    total_psnr = 0.0
    cnt = 0
    model.eval()
    with torch.no_grad():
        for iter_idx, data in enumerate(dataloader):

            input_img, label_img, name = data
            input_img = input_img.to(device)
            pred = net(input_img)

            pred_clip = torch.clamp(pred, 0, 1)
    
            pred_numpy = pred_clip.squeeze(0).cpu().numpy()
            label_numpy = label_img.squeeze(0).cpu().numpy()
            psnr = peak_signal_noise_ratio(pred_numpy, label_numpy, data_range=1)
            total_psnr += psnr
            cnt += 1
            
            if (iter_idx % 100 == 1):
                print(iter_idx)
                plt.imshow(np.transpose(pred_numpy, (1,2,0)))
                plt.show()
                plt.imshow(np.transpose(label_numpy, (1,2,0)))
                plt.show()
                print('%d iter PSNR: %.2f' % (iter_idx + 1, psnr))
            
        print('==========================================================')
        print('The average PSNR is %.2f dB' % (total_psnr / cnt))
    

In [10]:
class my_module3(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, 3, 1, 1)
        self.conv2 = nn.Conv2d(out_channel, out_channel, 3, 1, 1)
        self.bn = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.pool = nn.MaxPool2d(2,2)
        self.iden = nn.Conv2d(in_channel, out_channel, 1, 1, 0)

    def forward(self, x):
        identity = self.iden(x)
        out = self.conv1(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn(out)
        out += identity
        out = self.relu(out)
        out = self.pool(out)
        return out

In [11]:
class my_module5(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, 5, 1, 2)
        self.conv2 = nn.Conv2d(out_channel, out_channel, 5, 1, 2)
        self.bn = nn.BatchNorm2d(out_channel)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.pool = nn.MaxPool2d(2,2)
        self.iden = nn.Conv2d(in_channel, out_channel, 1, 1, 0)

    def forward(self, x):
        identity = self.iden(x)
        out = self.conv1(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.conv2(out)
        out = self.bn(out)
        out += identity
        out = self.relu(out)

        return out

In [12]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = my_module3(3,32) #112*112
        self.layer2 = my_module3(32,64) #56*56
        self.layer3 = my_module5(64,128) #56*56
        self.convT1 = nn.ConvTranspose2d(128,64,2,2,0) #112*112
        self.convT2 = nn.ConvTranspose2d(64,32,2,2,0) #224*224
        self.conv = nn.Conv2d(32, 3, 1, 1, 0)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.convT1(out))
        out = F.relu(self.convT2(out))
        out = self.conv(out)
        output = out + x
   
        return output

net = Net().to(device)

In [13]:
print(net)

Net(
  (layer1): my_module3(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (iden): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
  )
  (layer2): my_module3(
    (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (dropout): Dropout(p=0.3, inplace=False)
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (iden): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (layer3): my_modu

In [17]:
_train(net, max_epoch = 1000, lr = 0.002, resume=False)

Starting from Epoch : 1
Training complete in 3m 8s, Total time : 3m 8s
Epoch: 1, Loss: 0.168saved
Training complete in 2m 46s, Total time : 31m 6s
Epoch: 11, Loss: 0.162saved
Training complete in 3m 3s, Total time : 59m 26s
Epoch: 21, Loss: 0.161saved
Training complete in 2m 49s, Total time : 87m 36s
Epoch: 31, Loss: 0.161saved
Training complete in 2m 55s, Total time : 115m 30s
Epoch: 41, Loss: 0.160saved
Training complete in 2m 44s, Total time : 143m 35s
Epoch: 51, Loss: 0.159saved
Training complete in 2m 47s, Total time : 171m 8s
Epoch: 61, Loss: 0.159saved
Training complete in 2m 46s, Total time : 199m 24s
Epoch: 71, Loss: 0.160saved
Training complete in 2m 45s, Total time : 227m 13s
Epoch: 81, Loss: 0.157saved


KeyboardInterrupt: 

In [ ]:
_test(net)